In [ ]:
import sys
from pathlib import Path
import shutil
import json
import csv

import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

TMP = DIR/'tmp'
IN_DIR = TMP/'inputs'
OUT_DIR = TMP/'outputs'

%load_ext autoreload
%autoreload 2

# Start fresh
wt.rm_paths(TMP)


In [ ]:
# # Visually check that SRTM tiles cover NZ by 
# # making polygons and viewing them at https://geojson.io.

# tids = wt.SRTM_NZ_TILE_IDS
# polygons = wt.build_polygons(tids)
# collection = {'type': 'FeatureCollection', 'features': polygons}
# json.dumps(collection)

In [ ]:
# # Download SRTM data from NASA

# path = Path('/home/araichev/srtmnz/srtm1')
# url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL3.003/2000.02.11/"
# #url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/"
# tids = wt.SRTM_NZ_TILE_IDS
# for i, tid in enumerate(sorted(tids)):
#     filename = tid + '.SRTMGL3.hgt.zip'
#     href = url + filename
#     print(i, href)

In [ ]:
# # Download elevation test data

# TEST_DATA_DIR = DIR/'tests'/'data'
# ts = wt.read_transmitters(TEST_DATA_DIR/'transmitters.csv')[:2]
# lon_lats = wt.get_lon_lats(ts)
# tiles = wt.get_tile_names(lon_lats)
# print('Tiles to download =', tiles)
# wt.download_srtm(tiles, TEST_DATA_DIR/'srtm3', api_key=wt.get_secret('GITLAB_API_KEY'))

In [ ]:
high_definition = True

path = DIR/'tests'/'data'/'transmitters.csv'
transmitters = wt.read_transmitters(path)[:2]
for t in transmitters:
    print(t)

In [ ]:
# Download topography data

lonlats = wt.get_lonlats(transmitters)
tiles = wt.get_tile_ids(lonlats)
print('Tiles to download =', tiles)
if high_definition:
    path = IN_DIR/'srtm1'
else:
    path = IN_DIR/'srtm3'
api_key = wt.get_secret('GITLAB_API_KEY')

wt.download_srtm(tiles, path, high_definition=high_definition, api_key=api_key)

% ls {path}

In [ ]:
# Create SPLAT transmitter files

in_path = DIR/'tests'/'data'/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
if out_path.exists():
    shutil.rmtree(str(out_path))
    
wt.create_splat_transmitter_files(in_path, out_path)

% ls {out_path}

In [ ]:
# Create SPLAT topography files

if high_definition:
    in_path = IN_DIR/'srtm1'
else:
    in_path = IN_DIR/'srtm3'
out_path = OUT_DIR/'splat_files'

wt.create_splat_topography_files(in_path, out_path, high_definition=high_definition)

% ls {out_path}

In [ ]:
# Create coverage reports

transmitter_names = [t['name'] for t in transmitters[:1]]
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitter_names, 
  high_definition=high_definition)

% ls {out_path}

In [ ]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path)

% ls {path}